# Taming Our Future AI Overlords

Welcome! This is a workshop created for the [Lakeside Hackfest](https://www.lakeside-hackfest.com/) in 2018. Feel free to look around - but expect to learn things!

The goal is to get an overview over the current status of machine learning techniques in a way that enables you to keep learning and understand what people are talking about, at least roughly. Therefore, this session covers:

- Classic machine learning  
- Deep learning with Keras and CNTK/Tensorflow
- Cognitive Services 

In order to run this notebook, a Python 3.5 runtime environment is required, GPUs are of course preferred for more experimentation, but it should work with CPU only as well. With that available it's strictly necessary to have fun :) and simply work through everything from top to bottom. 


## Prepare

The following cell (clumsily) installs all required dependencies using the current Python's `pip` installer. If you have a new-ish NVidia GPU available, be sure to install `cntk-gpu` (or `tensorflow-gpu`) instead of their regular CPU versions. Check the `requirements.txt` file to find out more about the dependencies.


In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt  

## Classic Machine Learning

The first step is to try out a simple classic machine learning algorithm and see how it performs. This will also establish a baseline for later models to beat! The most common thing to use here is `scikit-learn`, and the problem we are tackling is a multiclass-classification problem: we want to know what category that clothing item belongs to!

### Task 1: >79% Accuracy

To understand the process better, let's try and **train a model to be more than 79% accurate** - still, every 5th classificaiton will be wrong, but it should be good enough for now. Find an algorithm and parameters from [here](http://scikit-learn.org/stable/modules/multiclass.html) and train a model that exceeds 79% accuracy! Try out different things:

- Algorithms
- Parameters
- Training data set size
- ...


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random


from sklearn import neighbors, linear_model, tree, ensemble, neural_network, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from collections import Counter

In [ ]:
def load_data(path):
    data = np.genfromtxt(path, delimiter=',')[1:] # skip header row
    y, x = np.split(data, [1], axis = 1) # split vertically after the first column, which contains the class
    return x, y

In [ ]:
def print_prediction_stats(label, y_pred, y_true):
    print(label, accuracy_score(y_true, y_pred))

In [ ]:
def train(model, X, y):
    model.fit(X, y)
    return model

In [ ]:
def explore(X, y):

    random_img = random.choice(X);
    plt.figure()
    plt.title("The Random Image")
    plt.imshow(random_img.reshape((28,28)), cmap = 'gray')
   
    plt.figure(figsize = (16, 6))
    plt.title("Random Image Histogram")
    c = Counter()
    for r in random_img: c[r] += 1
    colors = list(range(0, 256))
    plt.hist([c[h] for h in colors], colors)

In [ ]:
X_train, y_train = load_data("fashion-mnist_train.csv")
X_test, y_test = load_data("fashion-mnist_test.csv")

In [ ]:
def create_mnist(n_training = 5000):
    explore(X_train, y_train)
    X, _, y, _ = train_test_split(X_train, y_train, train_size=n_training, test_size=0)
    def f(model):
        trained = train(model, X, y)
        y_pred = trained.predict(X_test)
        print_prediction_stats(trained, y_test, y_pred)
        return trained
    return f

MNIST = create_mnist()

In [ ]:
#
# Task 1: Run MNIST(MyModelInstance)
#

# TODO: Code

## Deep Learning

In this second part we are going to utilize deep learning strategies in order to improve accuracy! In particular, the neural network we will use is a convolutional neural network with 18 layers, taken from the brilliant [deep Learning turkey](https://medium.com/deep-learning-turkey/deep-learning-lab-episode-1-fashion-mnist-c7af60029836). Instead of training from scratch (because who has that kind of time), we are going to use pre-learned weights to get a network that already performs very well. 

In fact, this is a very common strategy: taking existing network architectures ([AlexNet](https://en.wikipedia.org/wiki/AlexNet), VGGNet, LeNet, ...) and training on top of pre-trained weights (e.g. [ImageNet data](http://www.image-net.org/)). This process is also called transfer learning and is quite useful for getting good results quickly. 

Instead of the turkey's network we could have adjusted the image size (filling up the background) to conform to AlexNet's (or similar) dimensions of 224x224. This is something that you can try on your own.



### Task 2: Keep Training

Once a network has been created, it's possible to add weights from previous training runs giving it a head start. Use Keras to load weights into the model and train for another epoch or two (depending on the time).  

### Task 3: Gather Stats

With the deep learning model, predict the classes of a few images from the test set and see how it performs. To do that, collect classification errors relative to the sample size!


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow" # or cntk

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.regularizers import l2
from keras.datasets import fashion_mnist

In [ ]:
num_classes = 10


input_shape = (28, 28, 1)

x_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
x_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



In [ ]:

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=l2(0.01), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (5, 5), kernel_regularizer=l2(0.01)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(0.01)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (5, 5), kernel_regularizer=l2(0.01)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

model = build_model()
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
#
# Task 2: Keep training by loading the weights and running another training epoch or two.
#

# TODO: Insert code here

In [ ]:
#
# Task 3: Gather prediction accuracy for a few (10?) items from the test data set
#


sample_size = 20
error = 0

# TODO: Insert code here

print("Test accuracy: ", 1 - (error / sample_size))
    

## APIs

In a last attempt, let's try out some prebuilt services. In many cases they are a lot easier to use and produce usable results within minutes, making them ideal for prototypes as well as production cases. 

Microsoft's cognitive services contain various services around custom and prebuilt machine learning tasks, paid per call with a free tier. For this exercise, the free tier should work out fine - however production use cases better use a paid version :)

The service we are using here is called [custom vision](https://customvision.ai) and provides a trainable image classification service complete with: 

- Model versioning
- Rest API
- UI for labelling and data management


### Task 4: Data Wrangling

A major task in any machine learning engagement is to get the data you need in the shape, mode, and size required. With this task you should fill in the function `make_img()` which is expected to create an image from a simple array of values. This image should be usable for the API!

### Task 5: Test The API's Model

Once trained, the model can deliver predictions! Find out how, implement it, and see how well it performs!


In [ ]:
# Mapping class numbers to names

mapping = {
    0: "T-shirt/top", 
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}

training_key = ""
project_id = ""
prediction_key = ""

In [ ]:
from io import BytesIO
import time

from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models


In [ ]:
#
# Task 4: Create a JPEG, PNG, or BMP to upload to custom vision.
# 

def make_img(pd):
    # TODO: return an actual image
    return BytesIO()

In [ ]:
# Create API connection
trainer = training_api.TrainingApi(training_key)

# Creating tags twice will result in an error...
tags = [trainer.create_tag(project_id, name) for key, name in sorted(mapping.items(), key=lambda i: i[0])]

In [ ]:
# Generate a test set suitable for the free tier (5000 images take a long time to upload...)

X, _, Y, _ = train_test_split(X_train, y_train, train_size = 100, test_size=0)

for x, y in zip(X, Y):
    img_data = make_img(x)
    status = trainer.create_images_from_data(project_id, img_data, [ tags[np.where(y == 1)[0][0]].id ])    

In [ ]:
# Start training and wait for it to finish.

iteration = trainer.train_project(project_id)

while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project_id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)


In [ ]:
#
# Task 5: Predict! Check the tutorials for how to create a prediction endpoint and use it to send images there
#

sample_size = 20
errors = 0

# TODO: insert code here :)

print("Test accuracy: ", 1 - (errors / sample_size))

# Done!

That's it, there is way more to learn out there and this session should provide an overview over what's possible and a basic vocabulary. Check out these links to find out more about this entire space!

- [Andrew Ng's ML basics course](https://www.coursera.org/learn/machine-learning)
- [Deep Learning specialization](https://www.coursera.org/specializations/deep-learning)
- [Understanding CNNs](https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/)
- [Custom Vision SDK Tutorial](https://docs.microsoft.com/en-us/azure/cognitive-services/Custom-Vision-Service/python-tutorial)
- [Keras Docs](https://keras.io/)
- [Kaggle](https://kaggle.com)
- [Deep Learning the Fashion MNIST](https://medium.com/deep-learning-turkey/deep-learning-lab-episode-1-fashion-mnist-c7af60029836)
- [Fashion MNIST Benchmarks](http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/#)
- [CNTK Tutorials](https://cntk.ai/pythondocs/tutorials.html)
- [ML Tools on Azure](https://azure.microsoft.com/en-us/services/#ai-machine-learning)